In [4]:
import pandas as pd

d = pd.read_parquet('input/residents1.parquet')

df = d.dropna(subset=['LEIDIMO_DATA'])

In [6]:
def add_building_type(df, filepath):
    # Load the keyword and building type mappings from the CSV file
    mapping_df = pd.read_csv(filepath)

    # Define the function to categorize building type based on the "STATINIAI" column
    def categorize_building_type(statiniai):
        for _, row in mapping_df.iterrows():
            if pd.notna(statiniai) and row['keyword'] in statiniai:
                return row['building_type']
        return None  # Return None for cases that don't match any of the criteria

    # Apply the categorization function to the "STATINIAI" column
    df['building_type'] = df['STATINIAI'].apply(categorize_building_type)

    # Identify and print out the rows that didn't match any of the criteria
    unmatched = df[df['building_type'].isna()]['STATINIAI'].unique()
    if unmatched.size > 0:
        print("Unmatched 'STATINIAI' values that didn't fit into the criteria:")
        for value in unmatched:
            print(value)
    else:
        print("All 'STATINIAI' values have been categorized.")

    # Return the updated DataFrame
    return df


In [7]:
df=add_building_type(df,'building_types.csv')
df=df.dropna(subset=['building_type'])


Unmatched 'STATINIAI' values that didn't fit into the criteria:
 Komunalinių atliekų surinkimo aikštelė (Naujo statinio statyba)
 Prekybos paskirties pastatas (Naujo statinio statyba)
 Vidutinio slėgio dujotiekis (Naujo statinio statyba)
 Pastatas Nr. 2N (Naujo statinio statyba), Pastatas Nr. 4N (Naujo statinio statyba), Pastatas Nr. 3 (Naujo statinio statyba), Pastatas Nr. 2 (Naujo statinio statyba), Pastatas Nr. 4 (Naujo statinio statyba), Vandentiekio tinklai (Naujo statinio statyba), Buitinių nuotekų tinklai (Naujo statinio statyba), Pastatas Nr. 3N (Naujo statinio statyba), Pastatas Nr. 5 (Naujo statinio statyba), Pastatas Nr. 1N (Naujo statinio statyba), Pastatas Nr. 6N (Naujo statinio statyba), Pastatas Nr. 7N (Naujo statinio statyba), D2 ketegorijos gatvės dalis (Naujo statinio statyba), Paviršinių nuotekų tinklai (Naujo statinio statyba), Pastatas Nr. 1 (Naujo statinio statyba), Pastatas Nr. 7 (Naujo statinio statyba), Pastatas Nr. 8N (Naujo statinio statyba), Šilumos tiekimo 

C:\Users\austeja.burmonaite\AppData\Local\Temp\ipykernel_16888\3051128018.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_type'] = df['STATINIAI'].apply(categorize_building_type)


In [8]:
def add_building_time(df):
    # Convert the LEIDIMO_DATA and USER_registrationDate columns to datetime
    df['LEIDIMO_DATA'] = pd.to_datetime(df['LEIDIMO_DATA'], errors='coerce')
    df['USER_registrationDate'] = pd.to_datetime(df['USER_registrationDate'], errors='coerce')

    # Calculate the difference in days between USER_registrationDate and LEIDIMO_DATA
    df['building_time'] = (df['USER_registrationDate'] - df['LEIDIMO_DATA']).dt.days

    # Remove rows where building_time is negative or NaN
    df = df[df['building_time'].notna() & (df['building_time'] >= 0)]

    # Return the updated DataFrame
    return df

In [9]:
add_building_time(df)

,Join_Count,TARGET_FID,Join_Count_1,TARGET_FID_1,Join_Count_12,TARGET_FID_12,JOIN_FID,Match_addr,USER_gender,USER_dateOfBirth,...,Wetlands,Tree_Canopy,Meadow,Barren,Structures,Imperv_Surf,Imperv_Roads,Imperv_all,building_type,building_time
47,1,48.0,1,48.0,1.0,48,-1,"5 Aludarių g., Naujamiestis, Vilnius",Vyras,12/18/1926,...,0.0,264098377.0,202616940.0,0.0,340300484.0,151090870.0,40335608.0,49.0,daugiabutis,930.0
146,0,147.0,1,147.0,1.0,147,-1,"21D Kriaučiūnų g., Pilaitė, Vilnius",Vyras,3/18/1929,...,0.0,132086535.0,686574707.0,16210903.0,55090756.0,55629754.0,28902063.0,11.0,vienbutis,537.0
164,1,165.0,1,165.0,1.0,165,-1,"16A Putiniškių g., Pašilaičiai, Vilnius",Vyras,5/4/1929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vienbutis,953.0
257,1,258.0,1,258.0,1.0,258,-1,"3B Fabijoniškių g., Fabijoniškės, Vilnius",Vyras,3/30/1930,...,0.0,6121117.0,200599403.0,0.0,455531921.0,312038670.0,25699956.0,77.0,daugiabutis,1966.0
396,0,397.0,1,397.0,1.0,397,-1,"30E Rugiagėlių g., Naujoji Vilnia, Vilnius",Vyras,4/14/1931,...,0.0,270556736.0,491911163.0,0.0,165243683.0,49641070.0,20656536.0,21.0,dvibutis,367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633227,1,631032.0,1,631032.0,1.0,631032,-1,"72 Sasnos g., Pilaitė, Vilnius",Moteris,5/22/2024,...,0.0,374087639.0,320805092.0,12309786.0,140035906.0,127610989.0,25121543.0,27.0,dvibutis,928.0
633229,1,631034.0,1,631034.0,1.0,631034,-1,"1 Švitrigailos g., Naujamiestis, Vilnius",Moteris,5/24/2024,...,0.0,311639786.0,118461609.0,0.0,449636688.0,97813940.0,16985580.0,55.0,daugiabutis,2613.0
633235,1,631040.0,1,631040.0,1.0,631040,-1,"10 Salaspilio g., Fabijoniškės, Vilnius",Moteris,5/24/2024,...,1004840.0,0.0,84982061.0,763906021.0,9828349.0,49259720.0,82538948.0,8321346.0,daugiabutis,3577.0
633242,1,631047.0,1,631047.0,1.0,631047,-1,"12 Elbingo g., Pilaitė, Vilnius",Moteris,5/26/2024,...,0.0,47704630.0,586853600.0,34640012.0,165774784.0,160532570.0,4500891.0,33.0,daugiabutis,2239.0


In [10]:

def calculate_and_save_average_building_time(df, output_filepath):
    # Ensure that 'building_time' is calculated and valid (non-negative and non-NaN)
    df = add_building_time(df)

    # Remove rows where building_time is negative or NaN
    df = df[df['building_time'].notna() & (df['building_time'] >= 0)]

    # Group by 'building_type' and 'USER_eldership' and calculate the average 'building_time'
    result = df.groupby(['building_type', 'USER_eldership'])['building_time'].mean().reset_index()

    # Rename the 'building_time' column to 'average_building_time' for clarity
    result = result.rename(columns={'building_time': 'average_building_time'})

    # Save the result to a CSV file
    result.to_csv(output_filepath, index=False)

    # Return the resulting DataFrame
    return result


In [2]:
calculate_and_save_average_building_time(df,'permit_calculations.csv')

NameError: name 'df' is not defined